# Sources

In [ ]:
import os

from felt_python import (
    list_sources,
    create_source,
    get_source,
    update_source,
    delete_source,
    sync_source,
    create_map,
    add_source_layer,
    delete_map
)

os.environ["FELT_API_TOKEN"] = "<YOUR_API_TOKEN>"

# List available sources

In [ ]:
sources = list_sources()
sources

# Create a source

Below are examples for different source types. Uncomment the one you want to use.

In [ ]:
# Example 1: Create a PostgreSQL source
"""
postgres_source = create_source(
    name="My PostgreSQL Source",
    connection={
        "type": "postgresql",
        "host": "your-postgres-host.example.com",
        "port": 5432,
        "database": "your_database",
        "user": "your_username",
        "password": "your_password"
    },
    permissions={
        "type": "workspace_editors"  # Share with all workspace editors
    }
)
source_id = postgres_source["id"]
"""

# Example 2: Create a BigQuery source
"""
bigquery_source = create_source(
    name="My BigQuery Source",
    connection={
        "type": "bigquery",
        "project": "your-gcp-project",
        "dataset": "your_dataset",
        "credentials": "BASE64_ENCODED_SERVICE_ACCOUNT_JSON"
    },
    permissions={
        "type": "source_owner"  # Only accessible to you
    }
)
source_id = bigquery_source["id"]
"""

# Example 3: Create a WMS/WMTS source
wms_source = create_source(
    name="Public WMS Source",
    connection={
        "type": "wms_wmts",
        "url": "https://basemap.nationalmap.gov/arcgis/services/USGSTopo/MapServer/WMSServer"
    },
    permissions={
        "type": "workspace_editors"  # Share with all workspace editors
    }
)
source_id = wms_source["id"]
source_id

# Get source details

In [ ]:
source_details = get_source(source_id)
source_details

# Update a source

In [ ]:
updated_source = update_source(
    source_id=source_id,
    name="Updated WMS Source",
    # You can also update connection details if needed
    connection={
        "type": "wms_wmts",
        "url": "https://basemap.nationalmap.gov/arcgis/services/USGSTopo/MapServer/WMSServer"
    }
)
updated_source

# Synchronize a source

This triggers a synchronization of the source to update datasets.

In [ ]:
synced_source = sync_source(source_id)
synced_source

# Create a map and add a layer from the source

In [ ]:
# Create a map
map_resp = create_map(
    title="Map with source layers",
    lat=39.8283,
    lon=-98.5795,  # Center of USA
    zoom=4,
    public_access="private"
)
map_id = map_resp["id"]

# Wait for source synchronization to complete
import time
for i in range(60):  # Wait up to 5 minutes
    current_source = get_source(source_id)
    if current_source["sync_status"] == "completed":
        break
    print(f"Waiting for source sync... Status: {current_source['sync_status']}")
    time.sleep(5)

# List available datasets in the source
print(f"Available datasets: {len(current_source.get('datasets', []))}")
for dataset in current_source.get('datasets', [])[:5]:  # Show first 5 datasets
    print(f"- {dataset.get('name')} (ID: {dataset.get('id')})")

In [ ]:
# Choose a dataset from the source
if current_source.get('datasets'):
    dataset_id = current_source['datasets'][0]['id']
    
    # Add a layer from the source dataset
    layer_result = add_source_layer(
        map_id=map_id,
        source_layer_params={
            "from": "dataset",
            "dataset_id": dataset_id
        }
    )
    layer_result
    
# For SQL sources, you could use SQL query:
"""
sql_layer = add_source_layer(
    map_id=map_id,
    source_layer_params={
        "from": "sql",
        "source_id": source_id,
        "query": "SELECT * FROM your_table LIMIT 100"
    }
)
"""

# Clean up

Delete the map and source we created

In [ ]:
# Delete the map
delete_map(map_id)

# Delete the source
delete_source(source_id)

# Verify source is gone
sources_after = list_sources()
print(f"Source still exists: {any(s['id'] == source_id for s in sources_after)}")